# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-7B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集

5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：
```bash
pip install -r requirements.txt
```

In [1]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [2]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 从huggingface官网下载


# 从本地加载模型 7B
local_model_path = "/root/autodl-tmp/hf-models/DeepSeek-R1-Distill-Qwen-7B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
   # model_name = model_name,
    model_name = local_model_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.65 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [3]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [4]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [5]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [6]:
print(response[0].split("### Response:")[1])


<think>
好，我现在要处理这位患者的病例。他是一名28岁的程序员，最近一周工作到半夜，感觉头晕、脖子疼，有时候还恶心。首先，我需要考虑他的症状可能是什么原因引起的。

首先，他工作到半夜，长时间的高强度脑力劳动可能会导致疲劳，但他的症状包括头晕和恶心，这更像是一种神经相关的问题。可能他有睡眠问题，比如失眠或者睡眠质量差，导致大脑供血不足，出现头晕和恶心。

接下来，我应该考虑常见的神经系统疾病，比如高血压、偏头痛、 neck stiffness（颈部僵硬）或者可能是中风的早期症状。不过，中风通常需要更长时间的临床表现，而他只工作了一周，所以可能性较低。

然后，我会回想一下最近的诱因，比如他是否在工作过程中有使用电子设备过多，或者是否有摄入过多咖啡因或酒精。这些都可能导致神经症状的出现。

另外，我也需要考虑他的 neck position（头位），是否有不良的姿势习惯，比如长时间低头导致颈部肌肉紧张。这可能引起颈部疼痛和僵硬。

我还应该检查他的血压和血糖水平，因为这些指标在神经症状中也很重要。如果他有高血压，可能会导致头晕和恶心。

然后，我会建议他做一些初步的检查，比如头颈部的X光片，以排除任何结构性的问题，如颈椎病或脊髓压迫。如果他有明显的颈部僵硬或疼痛，可能需要进一步的评估。

此外，我会建议他记录他的症状，包括发作的频率、持续时间、是否有加重的情况，以及是否有伴随的其他症状，如视力模糊、面部麻木等。

最后，如果初步检查没有发现严重的问题，我会建议他进行一些放松训练，如深呼吸、冥想，或者调整工作姿势，以缓解症状。如果症状持续或加重，他可能需要进一步的医疗评估。
</think>

这位患者可能面临神经相关的问题，如颈椎病、偏头痛或睡眠障碍。建议进行头颈部检查和血压、血糖检测，记录症状并调整工作姿势，必要时寻求专业帮助。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh

![dataset](images/dataset.png)

In [7]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [8]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

# 指定本地数据集路径
local_dataset_path = "/root/autodl-tmp/hf-datasets/medical-o1-reasoning-SFT"

dataset = load_dataset(local_dataset_path, "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [9]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [10]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [11]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 3584, padding_idx=151654)
        (layers): ModuleList(
          (0-3): 4 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3584, out_features=3584, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3584, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3584, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [12]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [13]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 57 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (57 x 2 x 1) = 114
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 51 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.125200
2,3.029400
3,3.048000
4,3.049200
5,2.912400
6,2.889700
7,2.806900
8,2.739700
9,2.633700
10,2.503900


TrainOutput(global_step=60, training_loss=2.1819460868835447, metrics={'train_runtime': 1241.9688, 'train_samples_per_second': 3.092, 'train_steps_per_second': 0.048, 'total_flos': 1.5480430313314714e+17, 'train_loss': 2.1819460868835447})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [14]:
model.save_pretrained("qwen-1.5b_lora_model")

In [15]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

In [16]:
# 模型保存方式二选一（要么使用上面的分开保存，要么使用这里的合并 Lora 保存）
# model.save_pretrained_merged("qwen-1.5b_lora_model", tokenizer, save_method="merged_16bit")

### 9. 测试训练后的生成结果

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [18]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人已经吃了5天药，感觉腹痛稍微减轻，但还是有些发热，现在右下腹有个包块。我得想想怎么处理。

首先，我得考虑包块是什么。包块可能是个积液，也可能是个肿瘤。如果包块是积液，那可能是因为炎症扩散引起的。但如果是肿瘤，那就要特别小心了，因为肿瘤有可能是恶性的。

再想想发热，发热一般是因为感染或者炎症。急性阑尾炎通常会伴随发热，所以这个发热可能和炎症有关。不过，如果包块是肿瘤，那也得考虑一下是否有恶性的情况。

还有，包块的位置在右下腹，这个位置在阑尾附近，这可能是阑尾炎扩散到的区域，所以包块很可能是积液。但是，也有可能是肿瘤，尤其是如果包块比较大或有其他症状，比如疼痛或活动度变化。

现在，我得想想，包块可能是什么。如果包块是积液，那通常没有疼痛，而且容易被挤压。但如果是肿瘤，可能会有疼痛和活动度变化，特别是在右下腹。

所以，如果包块是积液，可能是个急性阑尾炎扩展的迹象，需要进一步检查，比如B超或者穿刺。但如果包块是肿瘤，可能需要考虑恶性的情况，这就要考虑手术治疗了。

不过，从目前的情况来看，病人已经有5天的急性阑尾炎，发热和包块，通常这个包块会是急性阑尾炎的延续，所以可能是积液。因此，建议进一步检查，看看包块是否是积液。

如果包块是积液，那应该做B超检查，看看是否有脓肿或者其他情况。如果B超显示积液，那继续处理急性阑尾炎的治疗就可以了。但如果有其他情况，比如包块变硬、疼痛或者活动度变化，就可能需要考虑肿瘤的可能性。

总的来说，尽管包块可能是积液，但最好还是再仔细观察一下，以确保没有其他问题，比如恶性肿瘤。这可能需要进一步的检查，比如B超和穿刺活检，以确定包块的性质，从而决定下一步的治疗。
</think>
根据目前的临床表现和病人的症状，右下腹的包块可能是急性阑尾炎扩散到的区域，通常表现为积液。为了进一步确认包块的性质，建议进行B超检查，以观察包块的形态和是否有脓肿。如果B超显示为积液，则继续观察和治疗急性阑尾炎，以缓解症状和控制感染。如果B超发现包块有其他异常情况，如变硬、疼痛或活动度改变，就应考虑包块可能为肿瘤的可能性，并进一步进行穿刺活检等进一步的诊断。这样可以确保我们能够准确地判断包块的性质，并采取相应的治疗措施。


In [19]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [20]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
60岁的男性患者出现右侧胸疼，这让我想到肺结核。肺结核是一种常见的慢性疾病，常常导致胸部疼痛。在这样的情况下，X线检查显示了右侧肋膈角消失，这进一步支持了肺结核的诊断。根据这个信息，肺结核可能是一个主要的诊断。

现在，患者还有右侧胸腔积液，这让我想到需要进一步的检查来了解胸水的性质。了解胸水的性质对于诊断和治疗至关重要。通常，了解胸水的性质会帮助我们判断它是否是脓液、脓胸或积液，这可能与肺结核有关。

在实验室检查中，血常规是基础，但可能无法直接提供足够的信息。超声检查可以提供关于胸水的形态、大小和结构的信息，但这可能不够详细。

实验室检查中，血常规可以帮助我们了解患者的总体健康状况，但可能与胸水的性质关系不大。如果我们要更深入地了解胸水的性质，可能需要进行其他实验室检查。

考虑到这些因素，我想到了胸水穿刺检查。胸水穿刺检查可以获取到胸水的样本，从而进行详细的化验，了解胸水的成分、细胞类型和细菌学特征。这可能是一个更直接和准确的手段，帮助我们了解胸水的性质。

此外，胸水穿刺检查还能帮助我们判断胸水的类型，例如是脓胸、脓液还是单纯积液。这在诊断和治疗中是重要的，因为不同的胸水性质可能需要不同的处理和治疗方案。

综上所述，胸水穿刺检查似乎是一个更直接和有助于了解胸水性质的实验室检查方式。它能够提供关于胸水成分和类型的详细信息，帮助我们做出更准确的诊断和治疗决定。
</think>
对于一名60岁的男性患者，出现右侧胸疼，并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液。在了解胸水的性质时，实验室检查中最直接和有助于提供详细信息的是胸水穿刺检查。通过胸水穿刺，可以获取胸水样本进行化验，了解胸水的成分、细胞类型和细菌学特征。此外，胸水穿刺还能帮助判断胸水的类型，如脓胸、脓液或单纯积液，这对于诊断和治疗非常重要。因此，胸水穿刺检查是了解胸水性质的最优实验室检查方式。


In [21]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个28岁的程序员突然感觉头晕，这让我想到他可能有贫血。因为经常熬夜，他可能缺铁，这会增加贫血的风险。不过，他的症状也有可能是由于贫血引起的。不过，我需要再想想，他最近是不是有其他症状呢？比如疲劳、心悸，这些也可能是贫血的表现。总之，贫血似乎是一个可能性。嗯，还有其他可能性吗？比如，他是不是有其他类型的贫血呢？比如，他是不是有叶酸缺乏，这会让他更容易出现这些症状？不过，叶酸缺乏通常是较长期的，而他的症状似乎比较突然。嗯，看来还是贫血的可能性比较大。总之，他可能有贫血，这是需要进一步检查确认的。嗯，如果他有贫血，那应该先做个血常规检查，看看铁蛋白和血红蛋白水平，这样可以更准确地判断。嗯，还有，他现在有没有什么其他症状呢？比如，心慌、头晕，这些也是贫血常见的表现。嗯，看来还是需要进一步确认。嗯，我觉得，如果他确实有贫血，那么他需要补充铁，比如吃含铁的食物，比如动物血红蛋白，或者是补铁片。嗯，补铁应该能帮助他改善这些症状。嗯，对了，他现在有没有其他症状呢？比如，他是不是还有其他贫血的迹象呢？嗯，看来还是需要进一步确认。嗯，总之，贫血的可能性很高，他应该先做个血常规检查，确认一下。嗯，如果他确实有贫血，那么他应该在医生的指导下进行治疗。嗯，看来，他需要进一步检查，看看是否还有其他可能的疾病，比如贫血或者其他类型的贫血，然后根据结果来决定下一步的治疗。
</think>
根据患者的症状，他28岁，是个程序员，长期熬夜，最近突然出现头晕、恶心等症状，这提示他可能有贫血。贫血的常见原因包括铁缺乏、叶酸缺乏、维生素B12缺乏等。由于患者长期熬夜，可能缺铁，这也是贫血的常见原因。然而，叶酸和维生素B12缺乏通常需要长期积累，而患者的症状看起来是突然出现，因此铁缺乏的可能性较大。为了确认这一点，建议进行血常规检查，包括检查铁蛋白和血红蛋白水平，以确认是否存在贫血。如果确认有贫血，可以考虑补充铁，比如吃含铁的食物或补铁片。此外，如果有必要，还可以进一步检查以排除其他可能的疾病。总之，患者很可能有贫血，建议尽快进行检查以明确病因，并在医生指导下进行治疗。
